In [1]:
import numpy as np
import pandas as pd
import os
import json
import spacy
from itertools import chain, count
from collections import Counter
import re
from pprint import pprint

In [2]:
dat = pd.read_pickle("../data/elmondo_es.pkl")

In [3]:
dat.head()

,headline,keyfacts,content,tags,time
http://www.elmundo.es/america/2014/01/02/52c4d39a22601d6f658b457c.html,Una jueza del Tribunal Supremo suspende parcia...,[ Decidió atender a los grupos conservadores y...,"El martes, antes de presidir la fiesta de fin ...",[],2014-01-02
http://www.elmundo.es/america/2014/01/02/52c4d99622601d6d658b458a.html,'La revolución cubana sigue sin compromisos co...,[ 'Jamás hemos cedido ni cederemos ante agresi...,El presidente Raúl Castro reveló que se está i...,[],2014-01-02
http://www.elmundo.es/america/2014/01/03/52c61ede268e3e3c528b456b.html,La NSA trabaja en un ordenador cuántico capaz ...,[ La información proviene de los documentos de...,La Agencia de Seguridad Nacional (NSA) trabaja...,[],2014-01-03
http://www.elmundo.es/america/2014/01/10/52cfbb62ca47415a218b456b.html,Último adiós a la ex Miss Venezuela Mónica Spe...,[ Mónica Spear y su marido fueron asesinados e...,Esta semana Venezuela ha recibido una noticia ...,[],2014-01-10
http://www.elmundo.es/america/2014/01/14/52d4b8ba268e3eb2318b456a.html,Michoacán pone en jaque al Gobierno de Peña Nieto,[ El Gobierno envía más policías y militares y...,La situación en el Estado mexicano de Michoacá...,[],2014-01-14


Check if there are any remaining articles without keyfacts:

In [4]:
(dat['keyfacts'].apply(len) == 0).sum()

0

Identify named entities:

In [5]:
nlp = spacy.load('es')

In [6]:
nlp.pipeline

In [7]:
fact_entlist = dat['keyfacts'].apply(lambda l: [nlp(s).ents for s in l])

In [8]:
story_entlist = dat['content'].apply(lambda x: nlp(x).ents)

## Testing Question Generation

In [9]:
pairs = []
ent_id = count()
num_id = count()
for i in range(5):
    doc_ents = np.array(story_entlist[i])
    for j in range(len(fact_entlist[i])):
        if len(fact_entlist[i][j]) == 0:
            pass
        else:
            for ent in fact_entlist[i][j]:
                matches = ~pd.isnull([re.search(r'(?<!\w)' + ent.text + r'(?!\w)', x.text) for x in doc_ents])
                if np.any(matches):
                    # remove all instances of ent (incl. partial matches) from the story entity list
                    nq_ents = doc_ents[~matches]
                    if ent.label_ in ['CARDINAL', 'ORDINAL']:
                        ent_type = 'number'
                        ent_iter = num_id
                    else:
                        ent_type = 'entity'
                        ent_iter = ent_id
                    ind = next(ent_iter)
                    # replace entity in question
#                     question_text = dat['keyfacts'][i][j].replace(ent.text, '@placeholder')
                    question_text = re.sub(r'(?<!\w)' + ent.text + r'(?!\w)', '@placeholder', dat['keyfacts'][i][j])
                    # replace entity in text
#                     content_text = dat['content'][i].replace(ent.text, '@{0}{1}'.format(ent_type, ind))
                    content_text = re.sub(r'(?<!\w)' + ent.text + r'(?!\w)', 
                                          '@{0}{1}'.format(ent_type, ind), dat['content'][i])
                    ans = '@{0}{1}'.format(ent_type, ind)
                    # replace other instances of entities
                    for other_ent in nq_ents:
                        if other_ent.label_ in ['CARDINAL', 'ORDINAL']:
                            ent_type = 'number'
                            ent_iter = num_id
                        else:
                            ent_type = 'entity'
                            ent_iter = ent_id
                        ind = next(ent_iter)
#                         content_text = content_text.replace(other_ent.text, '@{0}{1}'.format(ent_type, ind))
                        question_text = re.sub(r'(?<!\w)' + other_ent.text + r'(?!\w)', 
                                               '@{0}{1}'.format(ent_type, ind), question_text)
                        content_text = re.sub(r'(?<!\w)' + other_ent.text + r'(?!\w)', 
                                              '@{0}{1}'.format(ent_type, ind), content_text)
                    print(question_text + '?')
                    print('...')
                    print(ans)
                    print('...')
                    print(content_text)
                    print('--------------------')

 La información proviene de los documentos del ex analista @placeholder ?
...
@entity0
...
La @entity1 (@entity2) trabaja en la construcción de un ordenador cuántico que puede descifrar cualquier contraseña, incluso las de más alta seguridad, según reveló hoy en exclusiva el diario '@entity3', a partir de los documentos del ex analista de la @entity2 @entity0.
El desarrollo de la computación cuántica es un objetivo que persigue desde hace años la comunidad científica y en el que la @entity2, la @entity5 y @entity6 han hecho importantes avances en la @number0 década.
Un ordenador cuántico es mucho más rápido que @number1 común, tanto que es capaz de descifrar todas las formas de codificación, incluso las de más alta seguridad que se emplean para proteger secretos de Estado, transacciones financieras, e información médica y de negocios.
@entity7 los documentos proporcionados por @entity8, los trabajos de la @entity2 para construir un ordenador cuántico forman parte de un programa de inve

## Generate Questions

In [47]:
dat.shape

(15022, 5)

In [14]:
M = dat.shape[0]
pairs = []
ent_id = count()
num_id = count()
for i in range(M):
    if i % 500 == 0:
        print("generating question {0} to {1}...".format(i, i + 500))
    doc_ents = np.array(story_entlist[i])
    for j in range(len(fact_entlist[i])):
        if len(fact_entlist[i][j]) == 0:
            pass
        else:
            for ent in fact_entlist[i][j]:
                try:
                    matches = ~pd.isnull([re.search(r'(?<!\w)' + ent.text + r'(?!\w)', x.text) for x in doc_ents])
                except:
                    continue
                if np.any(matches):
                    # remove all instances of ent (incl. partial matches) from the story entity list
                    nq_ents = doc_ents[~matches]
                    if ent.label_ in ['CARDINAL', 'ORDINAL']:
                        ent_type = 'number'
                        ent_iter = num_id
                    else:
                        ent_type = 'entity'
                        ent_iter = ent_id
                    ind = next(ent_iter)
                    # replace entity in question
#                     question_text = dat['keyfacts'][i][j].replace(ent.text, '@placeholder')
                    question_text = re.sub(r'(?<!\w)' + ent.text + r'(?!\w)', '@placeholder', dat['keyfacts'][i][j])
                    # replace entity in text
#                     content_text = dat['content'][i].replace(ent.text, '@{0}{1}'.format(ent_type, ind))
                    content_text = re.sub(r'(?<!\w)' + ent.text + r'(?!\w)', 
                                          '@{0}{1}'.format(ent_type, ind), dat['content'][i])
                    ans = '@{0}{1}'.format(ent_type, ind)
                    # replace other instances of entities
                    for other_ent in nq_ents:
                        if re.search(r"\(|\)", other_ent.text):
                            continue
                        if other_ent.label_ in ['CARDINAL', 'ORDINAL']:
                            ent_type = 'number'
                            ent_iter = num_id
                        else:
                            ent_type = 'entity'
                            ent_iter = ent_id
                        ind = next(ent_iter)
#                         content_text = content_text.replace(other_ent.text, '@{0}{1}'.format(ent_type, ind))
                        try:
                            question_text = re.sub(r'(?<!\w)' + other_ent.text + r'(?!\w)', 
                                                   '@{0}{1}'.format(ent_type, ind), question_text)
                            content_text = re.sub(r'(?<!\w)' + other_ent.text + r'(?!\w)', 
                                                  '@{0}{1}'.format(ent_type, ind), content_text)
                        except:
                            print("skipped bad token {0}".format(other_ent.text))
                    pairs += [[question_text, ans, content_text]]

generating question 0 to 500...
generating question 500 to 1000...
generating question 1000 to 1500...
skipped bad token Abbas [
skipped bad token Abbas [
skipped bad token Abbas [
skipped bad token Belavezha [
generating question 1500 to 2000...
skipped bad token atentado contra Saleh [
skipped bad token atentado contra Saleh [
generating question 2000 to 2500...
skipped bad token La recompensa [
generating question 2500 to 3000...
skipped bad token Estado Islámico [
skipped bad token Estado Islámico [
skipped bad token Estado Islámico [
skipped bad token Muerte [
generating question 3000 to 3500...
skipped bad token Consejo Legislativo [
skipped bad token Consejo Legislativo [
skipped bad token Gobierno [
generating question 3500 to 4000...
skipped bad token Parlamento [
skipped bad token Parlamento [
skipped bad token IS [
skipped bad token IS [
skipped bad token IS [
generating question 4000 to 4500...
skipped bad token Al Sisi [
skipped bad token IS [
skipped bad token IS [
genera

In [15]:
df = pd.DataFrame(pairs, columns=["question", "answer", "story"])

In [16]:
df.head()

,question,answer,story
0,La información proviene de los documentos del...,@entity0,La @entity1 (@entity2) trabaja en la construcc...
1,@entity26 @entity19 y su marido fueron asesin...,@number10,Esta semana @entity11 ha recibido una noticia ...
2,Han detenido a 7 personas intregrantes de la ...,@entity30,Esta semana @entity31 ha recibido una noticia ...
3,@placeholder tiene una gran tasa de criminali...,@entity49,Esta semana @entity49 ha recibido una noticia ...
4,@entity64 tiene una gran tasa de criminalidad...,@number33,Esta semana @entity64 ha recibido una noticia ...


In [17]:
df.shape

(30297, 3)

In [58]:
df.to_csv("es_sample.csv")

In [18]:
df.to_pickle("../data/elmondo_es_qa.pkl")